## קוד מבוא

#### ספריות

In [1]:
import os
import sys
import pathlib
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### העלאת משתנים להרצת הקוד

In [3]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(future_folder)

## פונקציות

### פונקציות גלובליות

In [4]:
from global_functions import up_load_df, find_files_with_pattern, up_load_shp

##  העלאת טבלאות תחזית

מצב קיים

In [5]:
matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')
    
file = pathlib.Path(matching_files[0])
file_name = file.name
    
forecast_2020=up_load_df(forecast_version_folder_location, file_name[:-5])

מצב עתידי כל שנה

In [6]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

matching_files=find_files_with_pattern(r'{}\Intermediates'.format(software_data_folder_location), '_pop_2040_iplan')
file = pathlib.Path(matching_files[0])
v_date = file.name[:6]

year=['2025','2030','2035','2040','2045','2050']

for y in year: 
    folder_path=r'{}\Intermediates'.format(software_data_folder_location)
    file_name='{}_forecast_{}_full'.format(v_date,y)

    locals()['forecast_{}'.format(y)]=up_load_df(folder_path,file_name)

In [7]:
col_2050=['Taz_num','student_yeshiva',
 'student',
 'uni_students','aprt',
 'pop_without_dorms_yeshiva','emp_Education','emp_okev', 'emp_not_okev',
 'total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [8]:
col_2020=['Taz_num',
 'aprt_20','pop_without_dorms_yeshiva','student',
 'uni_students',
 'student_yeshiva','total_emp','emp_okev',
 'emp_not_okev','emp_Education',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

שכבת אזורי תנועה

In [9]:
# shp=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

taz=up_load_shp(r'{}\background_files\TAZ_V4_240404_with_geo_info.shp'.format(create_forecast_basic_software_location))

folder_path=r'{}\background_files'.format(software_data_folder_location)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

lst_change_sector=list(taz_sector_change.Taz_num)

shp.loc[(shp['Taz_num'].isin(lst_change_sector))&(shp['zonetype']=='Judea and Samaria'),'main_secto']='U_Orthodox'

col_taz=['Taz_num',
 'Taz_name',
 'main_secto',
 'Muni_Heb',
 'jeru_metro',
 'zonetype',
 'in_jerusal',
 'SCHN_NAME',]

חיבור הכל

In [10]:
df=shp[col_taz].merge(forecast_2020[col_2020],on='Taz_num',how='left')

שלב ראשון קיצרתי את השמות שלהם

In [11]:
for y in year: 
    locals()['forecast_{}'.format(y)]=locals()['forecast_{}'.format(y)][col_2050]

שלב שני לייצר לכל שנה רשימה חדשה של שמות השדות שלו עם שינוי

In [12]:
for y in year: 
    col=['student_yeshiva',
         'student',
         'uni_students','aprt',
         'pop_without_dorms_yeshiva','emp_Education','emp_okev', 'emp_not_okev',
         'total_emp',
         'agri',
         'Indus',
         'Com_hotel',
         'Business',
         'Public']
    
    x=[]
    
    for i in col: x=x+[i+'_{}'.format(y)]
        
    x=['Taz_num']+x
    
    locals()['col_{}'.format(y)]=x

שלב שלישי לקחת את השמות החדשים ולשנות את העמודות של כל אחד

In [13]:
for y in year: 
    locals()['forecast_{}'.format(y)].columns=locals()['col_{}'.format(y)]

שלב אחרון לחבר את כולם אל הקיים

In [14]:
for y in year: 
    df=df.merge(locals()['forecast_{}'.format(y)],on='Taz_num',how='left')

x='Public'
my_list = [x]
start_year = 2025
end_year = 2050
step = 5

for year in range(start_year, end_year + 1, step):
    text = x +'_'+ str(year)
    my_list.append(text)

print(my_list)


In [15]:
col_order=['zonetype',
 'jeru_metro',
 'Muni_Heb',
 'main_secto',
 'in_jerusal',
 'SCHN_NAME',
 'Taz_num',
 'Taz_name',
 'aprt_20',
 'aprt_2025',
 'aprt_2030',
 'aprt_2035',
 'aprt_2040',
 'aprt_2045',
 'aprt_2050',
 'pop_without_dorms_yeshiva',
 'pop_without_dorms_yeshiva_2025',
 'pop_without_dorms_yeshiva_2030',
 'pop_without_dorms_yeshiva_2035',
 'pop_without_dorms_yeshiva_2040',
 'pop_without_dorms_yeshiva_2045',
 'pop_without_dorms_yeshiva_2050',
 'student',
 'student_2025',
 'student_2030',
 'student_2035',
 'student_2040',
 'student_2045',
 'student_2050',
 'uni_students',
 'uni_students_2025',
 'uni_students_2030',
 'uni_students_2035',
 'uni_students_2040',
 'uni_students_2045',
 'uni_students_2050',
 'student_yeshiva',
 'student_yeshiva_2025',
 'student_yeshiva_2030',
 'student_yeshiva_2035',
 'student_yeshiva_2040',
 'student_yeshiva_2045',
 'student_yeshiva_2050',
 'total_emp',
 'total_emp_2025',
 'total_emp_2030',
 'total_emp_2035',
 'total_emp_2040',
 'total_emp_2045',
 'total_emp_2050',
 'emp_okev',
 'emp_okev_2025',
 'emp_okev_2030',
 'emp_okev_2035',
 'emp_okev_2040',
 'emp_okev_2045',
 'emp_okev_2050',
 'emp_not_okev',
 'emp_not_okev_2025',
 'emp_not_okev_2030',
 'emp_not_okev_2035',
 'emp_not_okev_2040',
 'emp_not_okev_2045',
 'emp_not_okev_2050',
 'emp_Education',
 'emp_Education_2025',
 'emp_Education_2030',
 'emp_Education_2035',
 'emp_Education_2040',
 'emp_Education_2045',
 'emp_Education_2050',
 'agri',
 'agri_2025',
 'agri_2030',
 'agri_2035',
 'agri_2040',
 'agri_2045',
 'agri_2050',
 'Indus',
 'Indus_2025',
 'Indus_2030',
 'Indus_2035',
 'Indus_2040',
 'Indus_2045',
 'Indus_2050',
 'Com_hotel',
 'Com_hotel_2025',
 'Com_hotel_2030',
 'Com_hotel_2035',
 'Com_hotel_2040',
 'Com_hotel_2045',
 'Com_hotel_2050',
 'Business',
 'Business_2025',
 'Business_2030',
 'Business_2035',
 'Business_2040',
 'Business_2045',
 'Business_2050',
 'Public',
 'Public_2025',
 'Public_2030',
 'Public_2035',
 'Public_2040',
 'Public_2045',
 'Public_2050']

In [16]:
save_file_path=r'{}'.format(forecast_version_folder_location)

save_excel_path=r'{}\{}_forecast_2020_till_2050_iplan.xlsx'.format(save_file_path,file_date)

df[col_order].to_excel(save_excel_path,index=False)